In [16]:
# load the vector files
import sys
import io
setting = "d2v"
Dataset = "pubmed"

vectorFilesDir = "../Data/"+Dataset+"/vectors/"+setting+"/"+setting+".txt"
allPaperVectors = []

with open(vectorFilesDir, 'r', encoding = 'utf8') as f:
    for line in f:
        read_data = line.split(" ")
        paper_Vectors = read_data
        allPaperVectors.append(paper_Vectors)
f.close()
        
print("Total vector records:",len(allPaperVectors))
print(allPaperVectors[0])

Total vector records: 3149075
['3', '-0.07245799', '-0.15048164', '-0.04320673', '0.01244448', '0.05051953', '-0.05573996', '0.03158288', '-0.04663554', '-0.00442508', '-0.02417533', '-0.03292065', '0.03798062', '0.08195730', '-0.09100581', '-0.04666801', '-0.06315092', '-0.05957321', '0.09766518', '0.01981102', '0.09956500', '-0.02059892', '-0.02321497', '0.10300557', '0.09654117', '0.02085607', '0.15179265', '0.03320639', '0.04716884', '0.04259005', '-0.01022485', '0.07371941', '0.02970656', '0.18967280', '0.07049462', '-0.07849123', '0.10272161', '0.05396378', '0.04138396', '0.08093689', '-0.04713648', '-0.08277001', '0.06004119', '0.15147503', '-0.10719796', '-0.06268646', '0.15823838', '0.10273122', '0.04453533', '-0.00394740', '-0.01239040', '-0.06826647', '-0.02995823', '0.14925463', '0.12254845', '-0.05894163', '0.11628735', '0.03898517', '0.01221054', '-0.00804257', '-0.06178775', '-0.04752085', '-0.04040224', '0.09192738', '0.01171173', '0.02951661', '-0.02156392', '-0.024588

In [17]:
import re
import numpy as np
import os

# collect data
fileDir = "../Data/"+Dataset+"/filteredSameNameAuthor/filter=10/"
fileList = os.listdir(fileDir)
fileList.sort()
print(fileList)


['alfredo martinez.txt', 'alfredo martinez0.txt', 'alfredo martinez1.txt', 'amit patel.txt', 'amit patel0.txt', 'amit patel1.txt', 'ana castro.txt', 'ana castro0.txt', 'ana castro1.txt', 'ana castro2.txt', 'anna ferrari.txt', 'anna ferrari0.txt', 'anna ferrari1.txt', 'bin liu.txt', 'bin liu0.txt', 'bin liu1.txt', 'carmen moreno.txt', 'carmen moreno0.txt', 'carmen moreno1.txt', 'carmen torres.txt', 'carmen torres0.txt', 'carmen torres1.txt', 'chao liu.txt', 'chao liu0.txt', 'chao liu1.txt', 'cheng luo.txt', 'cheng luo0.txt', 'cheng luo1.txt', 'chung-may yang.txt', 'chung-may yang0.txt', 'chung-may yang1.txt', 'david g lloyd.txt', 'david g lloyd0.txt', 'david g lloyd1.txt', 'fang liu.txt', 'fang liu0.txt', 'fang liu1.txt', 'feng liu.txt', 'feng liu0.txt', 'feng liu1.txt', 'feng xu.txt', 'feng xu0.txt', 'feng xu1.txt', 'francisco esteves.txt', 'francisco esteves0.txt', 'francisco esteves1.txt', 'francisco j blanco.txt', 'francisco j blanco0.txt', 'francisco j blanco1.txt', 'giovanni volpe

In [18]:
# collect class vectors
import pandas as pd
import numpy as np

def extractVectors(authors_pids, allPaperVectors):
    appended_data = []
    for label, author in enumerate(authors_pids):
        data = []
        labels = []
        for paper_Vectors in allPaperVectors:
            if paper_Vectors[0] in author:
                data.append(paper_Vectors)
                labels.append(label)
        print("Class ",label," sample size: ", len(data))
        # create df save one author data 
        authordf = pd.DataFrame(data)
        authordf['label'] = labels
        appended_data.append(authordf)
    # add all together
    dataset = pd.concat(appended_data, axis=0,ignore_index=True)
    # shuffle it
    dataset = dataset.sample(frac=1).reset_index(drop=True)
    # print shape for confirmation
    print(dataset.shape)
    return dataset

In [19]:
def spearate_train_test(data, ratio=0.5):
    label = data['label']
    pid = data[0]
    data = data.drop([0,'label'], axis=1)
    # take 50% test data
    test_data = data.sample(frac=ratio)
    test_label = label[test_data.index]
    
    train_data = data.drop(test_data.index)
    train_label = label[train_data.index]
    return train_data, train_label, test_data, test_label

In [21]:
from sklearn import svm
from sklearn import linear_model
from sklearn import metrics
from sklearn.decomposition import PCA

# loop through files in directory |
# add name to name list
# author as positive sample, other as all samples
name_list = []
# create name list for all authors have same name
for file in fileList:
    if not file.startswith('.'):
        if not re.match(r'\D*\d+.txt$', file):
            # fix the coding issue
            name_list.append(file.encode("utf-8", "surrogateescape").decode('utf8','surrogateescape')[:-4])
# print(name_list)

all_name = []
all_class_size = []
allprecision = []
allrecall = []
allaccuracy = []
allf1_micro = []
allf1_macro = []
allSampleSize = []

for name in name_list:
    # get author with same name together
    sameNameAuthorfile = []
    for file in fileList:
        file=file.encode("utf-8", "surrogateescape").decode('utf8','surrogateescape')
        if not file.startswith('.'):
            if re.match(r'\D*\d+.txt$', file):
                if name in file:
                    sameNameAuthorfile.append(file)
    n_class = (len(sameNameAuthorfile))
    print(name, " : " , sameNameAuthorfile, " Class size: ", n_class)
    # create array to save author's pid
    authors = [[] for i in range(0, n_class)]
    for index, same_name_author in enumerate(sameNameAuthorfile):
        # read author sample
        with open((fileDir+same_name_author).encode('utf-8'), 'r', encoding = 'utf8') as f:
            for line in f:
                authors[index].extend(line.strip().split(" "))
    print(authors)
    # extract vector of the authors
    data = extractVectors(authors, allPaperVectors)
    # get half of the data out as test data
    train_data, train_label, test_data, test_label= spearate_train_test(data, ratio=0.5)
    # now we have labels and vectors, just run it on classifier
    clf = linear_model.LogisticRegression(multi_class='multinomial', solver = 'saga')
    #clf = svm.SVC(kernel='linear', decision_function_shape='ovr')
    clf.fit(train_data, train_label)
#     # print the training scores
#     print("training score :", clf.score(train_data, train_label))
#     print("training score :", clf.score(test_data, test_label))
    # evalutation
    pred_label = clf.predict(test_data)
    accuracy = metrics.accuracy_score(test_label, pred_label)
    precision = metrics.precision_score(test_label, pred_label, average='macro')
    recall = metrics.recall_score(test_label, pred_label, average='macro')
    micro_f1 = metrics.f1_score(test_label, pred_label, average='micro')
    macro_f1 = metrics.f1_score(test_label, pred_label, average='macro')
    print("accuracy: ",accuracy, " micro_f1: ",micro_f1, " macro_f1: ",macro_f1)
    # save result to list
    all_name.append(name)
    all_class_size.append(n_class)
    allaccuracy.append(accuracy)
    allf1_micro.append(micro_f1)
    allf1_macro.append(macro_f1)
    allSampleSize.append(len(data))
    allprecision.append(precision)
    allrecall.append(recall)
# write evaluation result to excel
output = pd.DataFrame({'author name':all_name,"class_size":all_class_size,
                       "accuracy":allaccuracy,"mirco_f1":allf1_micro, "macro_f1":allf1_macro,
                       "macro_precision":allprecision,"macro_recall":allrecall, "TrainSampleCount":allSampleSize})
filename = "name_clf_LogisticRegression_"+setting+"_filter=10.csv"
output.to_csv("../result/"+Dataset+"/"+filename, encoding='utf-8',index=False)
print("Done")

alfredo martinez  :  ['alfredo martinez0.txt', 'alfredo martinez1.txt']  Class size:  2
[['21804608', '20605212', '19169274', '20143917', '17308048', '16322249', '15205337', '15016826', '15342420', '15450355', '12676952', '11927154', '12052835', '11116141', '11511519', '11318608', '8961247'], ['25281236', '23849253', '23478881', '22638789', '22885034', '20182928', '19471981', '19383131', '18344329', '18471274', '18349551', '17586670', '17160622', '12910327', '11312706', '11325924', '10933839', '10514259', '10440668', '10514255']]
Class  0  sample size:  17
Class  1  sample size:  20
(37, 102)
accuracy:  1.0  micro_f1:  1.0  macro_f1:  1.0
amit patel  :  ['amit patel0.txt', 'amit patel1.txt']  Class size:  2
[['23551637', '23154265', '22136410', '22763245', '22804815', '22465521', '22531636', '22317868', '21895958', '20105162', '20556785'], ['27554638', '27353018', '26768087', '27062902', '27061427', '26002107', '26686239', '26575034', '26173375', '24492407', '25394785', '25557304', '25

/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Class  0  sample size:  11
Class  1  sample size:  18
(29, 102)
accuracy:  0.8571428571428571  micro_f1:  0.8571428571428571  macro_f1:  0.8541666666666667
ana castro  :  ['ana castro0.txt', 'ana castro1.txt', 'ana castro2.txt']  Class size:  3
[['23666236', '22001288', '20435699', '19520721', '18474589', '10189169', '10189172', '9521057', '7927409', '8003610', '1418300'], ['27411333', '25355557', '25570446', '25570449', '25570682', '25571238', '24110586', '19339028', '19964490', '19163973', '19164038', '18002365', '18003203'], ['23296989', '22257026', '20031416', '19587930', '18082290', '18477512', '18788732', '17919914', '17708655', '17960923', '16249092', '16370929', '17105433', '15514991', '16183292', '16230018', '15780633', '16279768', '15674800', '15265494', '12735985', '12713836', '11881998', '12111750', '12369956', '11461758', '12369973', '11301412', '10966740', '10691694', '10819162', '11121617', '10966745', '10560739', '10197958', '10399487', '9871607', '9377087', '7796052']]

/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Class  0  sample size:  17
Class  1  sample size:  49
(66, 102)
accuracy:  0.9696969696969697  micro_f1:  0.9696969696969697  macro_f1:  0.9568627450980391
bin liu  :  ['bin liu0.txt', 'bin liu1.txt']  Class size:  2
[['26945049', '23645632', '23314820', '22350155', '21973320', '21983106', '21511871', '21514160', '20943427', '18437202', '18353984', '17905898', '16126864', '16113220'], ['24284804', '24244944', '24417359', '24437551', '23444831', '23223092', '23893174', '23742638', '23625815', '23184536', '23404950', '23860904', '23412737', '23957823', '24077521', '23836611', '23595726', '24221695', '23471733', '23359649', '24038281', '22544732', '22047010', '22964921', '22744642', '22508497', '22324725', '22878417', '22893375', '22495572', '22893564', '23043296', '23043485', '22784098', '22642547', '21960474', '21710990', '21710529', '21882808', '21351748', '21808781', '21442097', '21423951', '21816464', '21391695', '20864169', '21031384', '20376822', '20814922', '20491465', '20812231',

...

Class  0  sample size:  18
Class  1  sample size:  45
(63, 102)
accuracy:  0.8125  micro_f1:  0.8125  macro_f1:  0.7257142857142858
jacob john  :  ['jacob john0.txt', 'jacob john1.txt']  Class size:  2
[['26324078', '25803868', '25679505', '24976147', '24625279', '24666546', '24348143', '20460281', '20660972', '18284472', '11677538'], ['25091662', '25058664', '25018120', '25091663', '24629994', '24550609', '25161177', '25091689', '25091681', '23644312', '21277608', '21418447', '21793745', '20606950', '20606923', '17002729']]
Class  0  sample size:  11
Class  1  sample size:  16
(27, 102)
accuracy:  0.7857142857142857  micro_f1:  0.7857142857142857  macro_f1:  0.7846153846153847
jeong hwan kim  :  ['jeong hwan kim0.txt', 'jeong hwan kim1.txt']  Class size:  2
[['26021773', '24516703', '24914332', '25253962', '24696492', '24459646', '23425057', '23805155', '23323252', '23197889', '22099071', '21689141', '22669208', '22934534', '22133043', '22381530', '21728030', '22019793', '21525770', '

Class  0  sample size:  36
Class  1  sample size:  79
(115, 102)
accuracy:  0.9827586206896551  micro_f1:  0.9827586206896551  macro_f1:  0.9801573725624358
kyung su kim  :  ['kyung su kim0.txt', 'kyung su kim1.txt']  Class size:  2
[['23528677', '22695425', '22924652', '21982990', '22101201', '21722666', '21218029', '20825912', '20975550', '20825813', '19825222', '19996791', '19097734', '18660392'], ['24856736', '24553547', '23704754', '24981010', '24495814', '23632268', '22664745', '23648213', '23932734', '23603152', '23648567', '22867838', '22975022', '22867824', '23380094', '22683079', '23187987', '22487663', '22748697', '22980485', '21824465', '22227383', '22391544', '22463971', '21803673', '22533576', '22424993', '22657731', '22932993', '22322281', '20542398', '22372785', '21633052', '21908139', '21129911', '21610354', '20511643', '18996670', '21236547', '18722743', '19913985', '20599213', '20154545', '20837252', '19336669', '19542400', '19111959', '18638308', '19119441', '180227

Class  0  sample size:  16
Class  1  sample size:  98
Class  2  sample size:  141
(255, 102)
accuracy:  0.9375  micro_f1:  0.9375  macro_f1:  0.8409306742640075
mikael svensson  :  ['mikael svensson0.txt', 'mikael svensson1.txt']  Class size:  2
[['27401464', '26498954', '26271370', '27477892', '26093889', '26433734', '25070495', '22270230', '22175760', '23084655', '21924589', '21896369', '20012126', '20236746', '20441833', '19261344', '19393789', '17485154'], ['20624845', '22523602', '20602094', '19756356', '19290472', '18695542', '17103208', '17538371', '17415173', '16572654', '15918934', '15958504', '15805870', '16222210', '15165368', '14977478', '14577867', '14499639', '11780861', '11477600', '10222109', '10452353', '10585297', '10193622', '10521139', '9846834', '9187276', '8751634', '8981434', '7477922', '8360946', '8320000', '8261093', '8477825', '8457900', '1517400', '1393537', '1560254', '3338536', '3121377']]
Class  0  sample size:  18
Class  1  sample size:  40
(58, 102)
accu

Class  0  sample size:  15
Class  1  sample size:  102
(117, 102)
accuracy:  0.9827586206896551  micro_f1:  0.9827586206896551  macro_f1:  0.9497835497835498
vineet gupta  :  ['vineet gupta0.txt', 'vineet gupta1.txt']  Class size:  2
[['23386618', '23454649', '21804539', '21900205', '20188705', '20130528', '19879752', '19859549', '19067551', '18084301', '17438069', '17170130', '10404214', '10404215'], ['26704827', '26609018', '25821793', '23455328', '25530967', '24982692', '22475667', '22030127', '20856181', '19918470', '19192310', '19880010', '18294990', '17512845', '17630032', '17467502', '17382823', '17258665', '17434392', '17208631', '17113425', '16920464']]
Class  0  sample size:  14
Class  1  sample size:  22
(36, 102)
accuracy:  0.8333333333333334  micro_f1:  0.8333333333333334  macro_f1:  0.7339901477832513
vivek gupta  :  ['vivek gupta0.txt', 'vivek gupta1.txt']  Class size:  2
[['24269675', '24103331', '23364028', '22804843', '23873194', '22932477', '22709952', '22628913', '2

Class  0  sample size:  16
Class  1  sample size:  47
(63, 102)
accuracy:  0.90625  micro_f1:  0.90625  macro_f1:  0.8798498122653317
ying liu  :  ['ying liu0.txt', 'ying liu1.txt', 'ying liu2.txt']  Class size:  3
[['26900975', '25956638', '25559779', '25450926', '25388759', '25342538', '25051238', '22074855', '23017252', '21479784', '20444488', '19030987'], ['25893591', '26504151', '25581056', '26511265', '24737167', '25234843', '25261293', '23985142', '22983622', '22492774', '22484800', '21553295', '20686836', '21479687', '21390493', '20446031', '20160274', '20461020', '19936914'], ['24484300', '24479730', '24524252', '24166972', '23799926', '23627668', '23339573', '23577639', '23590568', '24074271', '24038597', '23135956', '22642669', '22549998', '22302415', '22644995', '21560170', '20507155', '20862713', '21050010', '19532885', '18851387']]
Class  0  sample size:  12
Class  1  sample size:  19
Class  2  sample size:  22
(53, 102)
accuracy:  0.9615384615384616  micro_f1:  0.9615384

In [ ]:
# 